In [74]:
import os
import re
import math
import time
import openai
import random
import pyttsx3
import elevenlabs
import audio_metadata
from moviepy.editor import *
from selenium import webdriver
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

In [75]:
with open('number_and_date.txt', 'r') as details:
    video_details = details.readlines()
short_no = int(video_details[0].split(' ')[0]) + 1
upload_date = datetime.strptime(video_details[0].split(' ')[-1][:-1], '%Y/%m/%d') + timedelta(days=1)
video_details.insert(0, str(short_no) + ' : ' +  upload_date.strftime('%Y/%m/%d') + '\n')
with open('number_and_date.txt', 'w') as details:
    details.writelines(video_details)
print(f'{short_no}th Video and {upload_date} schedule date has updated.')

39th Video and 2024-02-11 00:00:00 schedule date has updated.


In [76]:
api = open('api-key.txt', 'r').read()
openai.api_key = api

In [77]:
def getTopic():
    with open('topics.txt', 'r') as topics_file:
        first_topic = topics_file.readline()
    with open('completed_topics.txt', 'a') as completed_topics_file:
        completed_topics_file.write(first_topic)
    with open('topics.txt', 'r') as topics_file:
        remaining_topics = topics_file.readlines()[1:]
    with open('topics.txt', 'w') as topics_file:
        topics_file.writelines(remaining_topics)
    return first_topic

In [78]:
topic = getTopic()

In [79]:
def prompt():
    perfect_prompt = [
        f"I want short youtube video script for topic '{topic}'",
        '\nInitially it want to be start with question and give 3 steps to answer.',
        '\n25 seconds script is enough.',
        '\nLimit words around 70.'
        '\nI doing anotate video. I only need anotate sentences. give one paragarph. ',
        '\n\nexample:-\n',
        'Topic = Quick and Easy Hair Tutorials\n\n',
        'Answer:- Do you struggle with styling your hair?'+
        ' Well, you\'re in luck! Let\'t achieve a fabulous hairstyle in just a few steps.'+
        ' First, start by brushing your hair to remove any tangles.'+
        ' Then, create a high ponytail by gathering your hair at the crown of your head and securing it with an elastic.'+
        ' Finally, spritz your hair with some hairspray to hold the style in place all day. Voila!'+
        ' You\'re ready to rock a stylish and effortless look. So, why wait? Let\'s get started!',
        '\n\nI need above type of answer. only answer is enough.'
    ]
    return ''.join(perfect_prompt)

In [80]:
def gptOutput(content):
    responce = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": content}
        ]
    )
    print(responce['choices'][0]['message']['content'])
    return responce['choices'][0]['message']['content']

In [81]:
script = ''
generation_count = 0
while True:
    script = gptOutput(prompt())
    words = ['Topic:', 'Answer:', 'Question:']
    script_list = script.split()
    script_words = set(script.split())
    words_available = [word for word in words if word in script_words]
    if (len(words_available) == 0) and (len(script_list) < 140):
        break
    else:
        generation_count += 1
        if generation_count % 3 == 0:
            time.sleep(45)
        time.sleep(1)
        continue

Question: Are you looking for travel-friendly beauty essentials to pack?

Answer: Planning a trip but unsure of what beauty essentials to pack? Don't worry, we've got you covered! Follow these three steps to ensure you have all your travel-friendly beauty essentials ready to go. First, start by choosing multi-purpose products that can serve multiple functions. This will help save space in your luggage. Next, opt for travel-sized containers of your favorite products or invest in travel-sized toiletry bottles. These will comply with TSA regulations and save precious packing space. Finally, consider packing solid beauty products, such as solid perfume or shampoo bars, which are spill-proof and travel-friendly. By following these steps, you'll have all the beauty essentials you need for your journey without sacrificing space or convenience. Happy travels!
Do you find it difficult to pack your beauty essentials when traveling? No worries, we've got you covered! Here are three simple steps t

In [82]:
def description_prompt():
    description = script + '\n\n' + 'This is a 30 seconds youtube short video script. Give me a suitable description with relevent hashtags. I need only one paragraph.'
    return description

In [83]:
description = ''
des_count = 0
while True:
    description = gptOutput(description_prompt())
    words = ['Hashtags:']
    description_words = set(description.split())
    words_available = [word for word in words if word in description_words]
    if (len(words_available) == 0) or (des_count >= 2):
        break
    else:
        des_count += 1
        time.sleep(1)
        continue

"Struggling to pack your beauty essentials for your upcoming travels? Don't fret! Watch this quick and informative YouTube video for three foolproof steps to make your packing woes disappear. Discover how to choose multi-purpose products, find travel-sized versions of your favorites, and keep everything organized with handy packing cubes. With these practical tips, you'll save space in your luggage and still look fabulous wherever you go. Get ready to conquer your beauty packing game! #beautyessentials #traveltips #packinghacks"


In [84]:
with open(f'scripts/Short_{short_no}_script_and_description.txt', 'w') as file:
    paragraph ='Script:\n\n' + script + f'\n\nTitle :- {topic}' + '\n\nDescription:\n\n' + description
    file.write(paragraph)
print(f"Paragraph successfully saved to Short_{short_no}_script_and_description.txt")

Paragraph successfully saved to Short_39_script_and_description.txt


In [85]:
def count_letters(sentence):
    letters_count = sum(c.isalpha() for c in sentence)
    return letters_count

In [86]:
def separate_sentences(paragraph):
    sentences = []
    current_sentence = ""
    for char in paragraph:
        current_sentence += char
        if char in ['.', '!', '?']:
            sentences.append(current_sentence.strip())
            current_sentence = ""
    if current_sentence:
        sentences.append(current_sentence.strip())
    return sentences

def final_sentences():
    paragraph = script
    sentences_list = separate_sentences(paragraph)
    new_sentence_list = []
    temp_sentence = ''
    for sentence in sentences_list:
        temp_sentence += sentence + ' '
        if count_letters(temp_sentence)>=55:
            new_sentence_list.append(temp_sentence)
            temp_sentence = ''
    if temp_sentence:
        new_sentence_list.append(temp_sentence)
        temp_sentence = ''
    return new_sentence_list

In [87]:
def get_audio_length(file_path):
    metadata = audio_metadata.load(file_path)
    duration_in_seconds = metadata.streaminfo.duration
    return duration_in_seconds

In [103]:
voice_no = random.randint(1,8)

In [104]:
def voiceOver(text, countable=None, voice=1):
    lenght = 0.0
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[voice].id)
    if countable:
        engine.save_to_file(text, f'voices/Short_{short_no}_{countable}.mp3')
    else:
        engine.save_to_file(text, f'voices/Short_{short_no}.mp3')
    # engine.say(text)
    engine.runAndWait()
    if countable:
        lenght = get_audio_length(f'voices/Short_{short_no}_{countable}.mp3')
    else:
        lenght = get_audio_length(f'voices/Short_{short_no}.mp3')
    
    return lenght    

In [105]:
voice_over_ranges = []
final_sentences_list = final_sentences()
for s in final_sentences_list:
    i = final_sentences_list.index(s) + 1
    length_sub = voiceOver(s, countable=i, voice=voice_no)
    voice_over_ranges.append(length_sub)
    
final_voice_over_ranges = []
cumalative = 0
for i in range(len(voice_over_ranges)):
    t_l = (round(cumalative, 2) , round(cumalative + voice_over_ranges[i], 2), final_sentences_list[i])
    final_voice_over_ranges.append(t_l)
    cumalative += voice_over_ranges[i] + 0.1

In [106]:
voice_lenght = voiceOver(script, voice=voice_no)
voice_lenght

39.37378684807256

In [107]:
video_folder_path = 'D:/Python/Youtube Project/Project/videos/'
def get_random_video_file():
    all_files = os.listdir(video_folder_path)
    video_files = [file for file in all_files if file.endswith(('.mp4'))]
    if not video_files:
        print("No video files found in the folder.")
        return None
    while True:
        random_video = random.choice(video_files)
        with open('video_log.txt', 'r') as video_log:
            videos = video_log.readlines()
        recent_videos = videos[:10]
        if (random_video + '\n') in recent_videos:
            continue
        else:
            videos.insert(0, random_video + '\n')
            break
    with open('video_log.txt', 'w') as video_log:
        video_log.writelines(videos)
    return os.path.join(video_folder_path, random_video)

In [108]:
audio_folder_path = 'D:/Python/Youtube Project/Project/Background_Musics/'
def get_random_audio_file():
    all_files = os.listdir(audio_folder_path)
    audio_files = [file for file in all_files if file.endswith(('.mp3'))]
    if not audio_files:
        print("No audio files found in the folder.")
        return None
    random_audio = random.choice(audio_files)
    return os.path.join(audio_folder_path, random_audio)

In [109]:
def remakeSentence(sentence):
    letter_limit = 18
    result = count_letters(sentence)
    words = sentence.split(' ')
    final_output = ''
    for i in range(math.ceil(result/letter_limit)):
        segment = ''
        while count_letters(segment)<letter_limit:
            if len(words)==0:
                break
            segment += words.pop(0) + ' '
        if (i == math.ceil(result/letter_limit) - 1) or len(words)==0:
            final_output += segment
        else:
            final_output += segment + '\n'
    return final_output

In [110]:
video_lenght = 0
clips = []
while video_lenght<=voice_lenght:
    clip = VideoFileClip(get_random_video_file())
    duration = clip.duration
    if (duration > voice_lenght) and len(clips)==0:
        clip2 = clip.subclip(0, math.ceil(voice_lenght)).fx(vfx.fadein, 0.5).fx(vfx.fadeout, 0.5)
        video_lenght += math.ceil(voice_lenght) 
        clips.append(clip2)
    elif duration > (voice_lenght - video_lenght):
        clip2 = clip.subclip(0, math.ceil(voice_lenght-video_lenght)).crossfadein(1).fx(vfx.fadeout, 0.5)
        video_lenght += clip2.duration
        clips.append(clip2)
    elif len(clips)==0:
        clip2 = clip.fx(vfx.fadein, 0.5)
        video_lenght += duration
        clips.append(clip2)
    else:
        clip2 = clip.crossfadein(1)
        video_lenght += duration
        clips.append(clip2)
selected = AudioFileClip(get_random_audio_file())
initial_point = random.randint(0, math.floor(selected.duration - video_lenght))
background = selected.subclip(initial_point, initial_point + video_lenght).fx(afx.audio_fadein, 0.5).fx(afx.volumex, 0.075)
voice_clip = AudioFileClip(f'voices/Short_{short_no}.mp3')

text_clips = []
for start, end, prase in final_voice_over_ranges:
    sentence = remakeSentence(prase)
    text_clip = TextClip(sentence, fontsize=60, color='yellow', font='Montserrat', kerning=1).set_position('center')
    tc_w, tc_h = text_clip.size
    color_clip = ColorClip(size=(tc_w+50, tc_h+10), color=(0,0,0)).set_opacity(0.25)
    middle_clip = CompositeVideoClip([color_clip, text_clip])
    middle_clip = middle_clip.set_position(('center', (1440-tc_h/2))).set_start(start).set_end(end).fx(vfx.fadein, 0.25).fx(vfx.fadeout, 0.25)
    text_clips.append(middle_clip)

combined_video_clips = concatenate_videoclips(clips, method='compose')
combined = CompositeVideoClip([combined_video_clips] + text_clips)
combined.audio = CompositeAudioClip([background, voice_clip])
combined.write_videofile(f'Shorts/Short_{short_no}.mp4', codec='h264_nvenc', threads=4)

Moviepy - Building video Shorts/Short_39.mp4.
MoviePy - Writing audio in Short_39TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Shorts/Short_39.mp4



Moviepy - Done !
Moviepy - video ready Shorts/Short_39.mp4


In [111]:
default_keywords = ' #beautyisthekey #beautytutorial #beautyhacks #beautyinspiration #beautysecrets #beautyproducts #beautyandstyle #beautycare #makeupartist #beautyblogger #beautytips #beautyaddict #makeuplook #beautycommunity #beautycare #beautyful'

In [112]:
options = Options()
profile = webdriver.FirefoxProfile('C:\\Users\\ASUS\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\plgll2zq.default-release')
options.profile = profile
driver = webdriver.Firefox(options=options)
driver.get("https://studio.youtube.com/")
time.sleep(3)

upload_button = driver.find_element(By.XPATH, '//*[@id="upload-icon"]')
upload_button.click()

file_input = driver.find_element(By.XPATH, '//*[@id="content"]/input')
simp_path = f'Shorts/Short_{short_no}.mp4'
abs_path = os.path.abspath(simp_path)
file_input.send_keys(abs_path)
time.sleep(5)

text_input = driver.find_element(By.XPATH, '//*[@id="title-wrapper"]//*[@id="textbox"]')
title = topic[:-1]
text_input.clear()
text_input.send_keys(title)
time.sleep(1)

description_input = driver.find_element(By.XPATH, '//*[@id="description-textarea"]//*[@id="textbox"]')
description = description.split('Description: ')[-1] + default_keywords
description_input.clear()
description_input.send_keys(description)
time.sleep(1)

child_verification = driver.find_element(By.XPATH, '//*[@id="audience"]/ytkc-made-for-kids-select/div[4]/tp-yt-paper-radio-group/tp-yt-paper-radio-button[2]')
child_verification.click()
time.sleep(1)

next_button = driver.find_element(By.XPATH, '//*[@id="next-button"]')
for i in range(3):
    next_button.click()
    time.sleep(1)
time.sleep(1)

schedule_select = driver.find_element(By.XPATH, '//*[@id="schedule-radio-button"]')
schedule_select.click()
time.sleep(1)

schedule_trigger = driver.find_element(By.XPATH, '//*[@id="datepicker-trigger"]/ytcp-dropdown-trigger/div')
schedule_trigger.click()

move_next = False
move_further = False
currunt_date = datetime.strptime(''.join(schedule_trigger.text.split(',')), '%b %d %Y')
if (upload_date.month - currunt_date.month) == 1:
    move_next=True
elif (upload_date.month - currunt_date.month) == 2:
    move_next = True
    move_further = True
time.sleep(1)

spans = driver.find_elements(By.XPATH, '//*[@id="calendar"]//span')
for span in spans:
    if span.text == str(upload_date.day) and move_next and move_further:
        driver.find_element(By.XPATH, '//*[@id="control-area"]//*[@id="next-month"]').click()
        time.sleep(1)
        move_further = False
        continue
    elif span.text == str(upload_date.day) and move_next:
        driver.find_element(By.XPATH, '//*[@id="control-area"]//*[@id="next-month"]').click()
        time.sleep(1)
        move_next = False
        continue
    elif span.text == str(upload_date.day):
        span.click()
        break
time.sleep(1)

schedule_time = driver.find_element(By.XPATH, '//*[@id="time-of-day-container"]')
schedule_time.click()
time.sleep(1)
selected_time = driver.find_element(By.XPATH, '//*[@id="dialog"]/tp-yt-paper-listbox/tp-yt-paper-item[23]')
selected_time.click()
time.sleep(10)

done_button = driver.find_element(By.XPATH, '//*[@id="done-button"]')
done_button.click()
time.sleep(30)

driver.close()
print(f'{short_no}th Video uploaded Successfully for {upload_date} date.')

39th Video uploaded Successfully for 2024-02-11 00:00:00 date.
